In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 4.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=385ce1357097a985412cab5ce2200dc4630027340b27f279633ab9f9b7d406fc
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=09edfd71aa398503942821a3b01c79aac5cd8651e3e9849a715ffee6db350db1
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('con_1_filter',min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                              activation='relu',
                              input_shape=(28,28,1)
                          ),
                           keras.layers.Conv2D(
                              filters=hp.Int('con_1_filter',min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                              activation='relu'
                        
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int('dense_1_unit',min_value=32,max_value=128,step=16),
                              activation='relu'
                          ),
                          
keras.layers.Dense(10,activation='softmax')


  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                
                )
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

In [12]:
tuner_search=RandomSearch(build_model,
                           objective='val_accuracy',
                           max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 16m 04s]
val_accuracy: 0.8306666612625122

Best val_accuracy So Far: 0.8548333048820496
Total elapsed time: 01h 52m 57s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_______________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_______________________
conv2d_1 (Conv2D)            (None, 20, 20, 96)        230496    
_______________________
flatten (Flatten)            (None, 38400)             0         
_______________________
dense (Dense)                (None, 96)                3686496   
_______________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,920,458
Trainable params: 3,920,458
Non-trainable params: 0
_______________________
